# 📥 Descargar y preparar dataset Dogs vs Cats desde Kaggle


In [8]:
# 📌 Celda: Descargar y preparar dataset Cats vs Dogs desde Kaggle



# 2️⃣ Descargar el dataset usando la API de Kaggle
# Se guardará en la carpeta "data/"
#!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset -p data/

# 📌 Celda: Filtrar imágenes corruptas y crear DataLoaders

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import os

# Función para verificar imágenes válidas
def remove_invalid_images(dataset_dir):
    removed_count = 0
    for root, _, files in os.walk(dataset_dir):
        for file in files:
            path = os.path.join(root, file)
            try:
                img = Image.open(path)
                img.verify()  # Verifica que la imagen sea válida
            except (IOError, SyntaxError):
                os.remove(path)  # Elimina imagen corrupta
                removed_count += 1
    print(f"Se eliminaron {removed_count} imágenes corruptas del dataset.")

# Directorio del dataset
full_dataset_dir = os.path.join(extract_path, "PetImages")  # Ajustar según tu estructura

# 1️⃣ Eliminar imágenes corruptas
remove_invalid_images(full_dataset_dir)

# 2️⃣ Transformaciones
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # Media ImageNet
                         [0.229, 0.224, 0.225])   # Std ImageNet
])

# 3️⃣ Crear dataset completo
full_dataset = datasets.ImageFolder(full_dataset_dir, transform=transform)

# 4️⃣ Dividir en train y validation (80/20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 5️⃣ Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Imágenes de entrenamiento: {len(train_dataset)}, Validación: {len(val_dataset)}")



Se eliminaron 3 imágenes corruptas del dataset.
Imágenes de entrenamiento: 19998, Validación: 5000


/home/miningdox/stable-diffusion-webui/venv/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


# 📌 Celda: Transformaciones y DataLoaders

Se preparan las imágenes para el modelo preentrenado:

- Redimensionamos a 224x224 (requisito de ResNet50)
- Normalizamos usando la media y desviación estándar de ImageNet
- Creamos datasets y DataLoaders para entrenamiento y validación


In [9]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transformaciones
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # Media ImageNet
                         [0.229, 0.224, 0.225])   # Std ImageNet
])

# Directorio del dataset
full_dataset_dir = os.path.join(extract_path, "PetImages")  # Ajustar según estructura

# Crear dataset completo
full_dataset = datasets.ImageFolder(full_dataset_dir, transform=transform)

# Dividir en train y validation (80/20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Imágenes de entrenamiento: {len(train_dataset)}, Validación: {len(val_dataset)}")


Imágenes de entrenamiento: 19998, Validación: 5000


# 📌 Celda: Configurar ResNet50 para Transfer Learning

- Cargamos ResNet50 preentrenada
- Congelamos las capas base
- Reemplazamos la última capa fully connected (fc) para 2 clases (gato vs perro)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device("cpu")
print("Usando dispositivo:", device)

# Cargar ResNet50 preentrenada
resnet = models.resnet50(pretrained=True)

# Congelar todas las capas base
for param in resnet.parameters():
    param.requires_grad = False

# Reemplazar capa final para 2 clases
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 2)  # gato o perro

resnet = resnet.to(device)

# Pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)  # solo entrenamos fc


Usando dispositivo: cpu


# 📌 Celda: Funciones de entrenamiento y validación


In [11]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    return running_loss / total, correct / total

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            running_loss += loss.item() * x.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return running_loss / total, correct / total


# 📌 Celda: Entrenamiento del modelo

- Entrenamos durante varias épocas
- Mostramos pérdida y accuracy para entrenamiento y validación


In [12]:
num_epochs = 2
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(resnet, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(resnet, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}\n")


KeyboardInterrupt: 

# 📌 Celda: Visualizar predicciones del modelo

Mostramos algunas imágenes del conjunto de validación con su predicción.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

classes = full_dataset.classes

def imshow(inp, title=None):
    inp = inp.numpy().transpose((1,2,0))
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title:
        plt.title(title)
    plt.axis('off')

# Mostrar algunas predicciones
resnet.eval()
inputs, labels = next(iter(val_loader))
inputs, labels = inputs.to(device), labels.to(device)
outputs = resnet(inputs)
_, preds = torch.max(outputs, 1)

plt.figure(figsize=(12,6))
for i in range(8):
    plt.subplot(2,4,i+1)
    imshow(inputs[i].cpu(), title=f"Pred: {classes[preds[i]]}\nTrue: {classes[labels[i]]}")
plt.show()
